In [ ]:
import time
import multiprocessing as mp
from sympy import symbols, Integer, expand

# ---------------------------------------------------------------------
# Symbols and weights 
# ---------------------------------------------------------------------
q, e, f, g, h = symbols('q e f g h')
#We use the notation 

#p_{a\mid bc} = e
#p_{b\mid ac} = f
#p_{abc}= g
#p_{a\mid b \mid c} = h

w = [e, f, g, h]
v = [w, w, w, w, w, w]   # length = 6

# ---------------------------------------------------------------------
# Hyperedges (12 triples), duplicated "upper" by +10
# ---------------------------------------------------------------------
data_lower = [
    [2, 1, 0],
    [2, 5, 6],
    [7, 6, 8],
    [4, 1, 3],
    [7, 5, 3],
    [4, 9, 8]
]
data = data_lower + [[x + 10 for x in trio] for trio in data_lower]  # length 12

# Precompute 5^i for i = 0..11
POW5 = [5**i for i in range(12)]

# ---------------------------------------------------------------------
# Union-Find (DSU) over n=20 vertices 
# ---------------------------------------------------------------------
def make_dsu(n=20):
    parent = list(range(n))
    rank = [0]*n
    return parent, rank

def find_root(parent, x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(parent, rank, a, b):
    ra = find_root(parent, a)
    rb = find_root(parent, b)
    if ra == rb:
        return
    if rank[ra] < rank[rb]:
        parent[ra] = rb
    elif rank[ra] > rank[rb]:
        parent[rb] = ra
    else:
        parent[rb] = ra
        rank[ra] += 1

# ---------------------------------------------------------------------
# Worker: process_subchunk(start, SUBCHUNK, TOTAL) -> SymPy expression
# ---------------------------------------------------------------------
def process_subchunk(args):
    start, SUBCHUNK, TOTAL = args
    END = min(start + SUBCHUNK, TOTAL)
    local_sum = Integer(0)

    for idx in range(start, END):
        # Decode idx into a 12‐tuple base[] in {1..5}
        b0  = (idx // POW5[0])  % 5 + 1
        b1  = (idx // POW5[1])  % 5 + 1
        b2  = (idx // POW5[2])  % 5 + 1
        b3  = (idx // POW5[3])  % 5 + 1
        b4  = (idx // POW5[4])  % 5 + 1
        b5  = (idx // POW5[5])  % 5 + 1
        b6  = (idx // POW5[6])  % 5 + 1
        b7  = (idx // POW5[7])  % 5 + 1
        b8  = (idx // POW5[8])  % 5 + 1
        b9  = (idx // POW5[9])  % 5 + 1
        b10 = (idx // POW5[10]) % 5 + 1
        b11 = (idx // POW5[11]) % 5 + 1
        base = (b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11)

        parent, rank = make_dsu(20)
        mon = Integer(1)

        # “Post” edges: union(2,12), union(4,14), union(7,17)
        for p_ in (2, 4, 7):
            union(parent, rank, p_, p_ + 10)
        
        # Add edges per base[i] and multiply v[i%6]
        for i, b in enumerate(base):
            a0, a1, a2 = data[i]
            if b == 1:
                union(parent, rank, a1, a2)
                mon *= v[i % 6][0]
            elif b == 2:
                union(parent, rank, a1, a0)
                mon *= v[i % 6][1]
            elif b == 4:
                union(parent, rank, a0, a1)
                union(parent, rank, a0, a2)
                mon *= v[i % 6][2]
            elif b == 3:
                union(parent, rank, a2, a0)
                mon *= v[i % 6][1]
            else:  # b == 5
                mon *= v[i % 6][3]

        # factor = q^(number_of_components)
        roots = { find_root(parent, x) for x in range(20) }
        mon *= q**len(roots)

        # If root(0)==root(9), add; if root(0)==root(19), subtract
        if ((find_root(parent, 0) == find_root(parent, 9)) and (find_root(parent, 0) != find_root(parent, 19))):
            local_sum += mon
        elif ((find_root(parent, 0) == find_root(parent, 19)) and (find_root(parent, 0) != find_root(parent, 9))):
            local_sum -= mon

    return local_sum

# ---------------------------------------------------------------------
# Orchestration
# ---------------------------------------------------------------------
if __name__ == "__main__":
    # TOTAL = 5^12 (244,140,625); SUBCHUNK = 5000
    TOTAL = 5**12
    SUBCHUNK = 5_000

    starts = list(range(0, int(TOTAL), SUBCHUNK))
    NCHUNKS = len(starts)

    print("TOTAL configurations:", TOTAL)
    print("SUBCHUNK size:", SUBCHUNK, "→", NCHUNKS, "chunks")

    # Prefer 'fork' when available (Linux/macOS). Falls back to 'spawn' if needed.
    try:
        ctx = mp.get_context("fork")
        start_method = "fork"
    except ValueError:
        ctx = mp.get_context("spawn")
        start_method = "spawn"

    print("Using", mp.cpu_count(), f"cores for parallel processing (start method: {start_method}).")

    start_time = time.time()

    # Launch the parallel jobs: each returns its local_sum 
    with ctx.Pool(processes=mp.cpu_count()) as pool:
        args_iter = ((s, SUBCHUNK, TOTAL) for s in starts)
        jobs = pool.imap_unordered(process_subchunk, args_iter, chunksize=8)

        total_poly = Integer(0)
        count = 0
        for partial in jobs:
            total_poly += partial
            count += 1

            # Print a status line every 1000 chunks 
            if count % 1000 == 0 or count == NCHUNKS:
                
                elapsed = time.time() - start_time
                print(f"  → Completed chunk {count}/{NCHUNKS} in {elapsed:.1f}s")

    elapsed_total = time.time() - start_time
    print("ALL DONE in %.1f seconds" % elapsed_total)
    print("FINAL SYMBOLIC POLY =")
    print(total_poly)


TOTAL configurations: 244140625
SUBCHUNK size: 5000 → 48829 chunks
Using 16 cores for parallel processing (start method: fork).
  → Completed chunk 100/48829 in 28.3s
  → Completed chunk 200/48829 in 50.6s
  → Completed chunk 300/48829 in 72.9s
  → Completed chunk 400/48829 in 97.0s
  → Completed chunk 500/48829 in 117.6s
  → Completed chunk 600/48829 in 139.0s
  → Completed chunk 700/48829 in 160.9s
  → Completed chunk 800/48829 in 174.2s
  → Completed chunk 900/48829 in 197.5s
  → Completed chunk 1000/48829 in 215.0s
  → Completed chunk 1100/48829 in 236.5s
  → Completed chunk 1200/48829 in 258.8s
  → Completed chunk 1300/48829 in 278.0s
  → Completed chunk 1400/48829 in 301.5s
  → Completed chunk 1500/48829 in 323.5s
  → Completed chunk 1600/48829 in 337.4s
  → Completed chunk 1700/48829 in 358.8s
  → Completed chunk 1800/48829 in 369.9s
  → Completed chunk 1900/48829 in 386.0s
  → Completed chunk 2000/48829 in 408.7s
  → Completed chunk 2100/48829 in 427.5s
  → Completed chunk 2200

In [ ]:
import pickle 
pickle.dump(total_poly, open("./poly", "wb"))

In [ ]:
total_poly = pickle.load( open("./poly", "rb"))

In [6]:
def print_wrapped_text(text, line_length=100):
    """
    Prints the given text, wrapping it to a specified line length.

    Args:
        text (str): The input string to be printed.
        line_length (int): The maximum number of characters per line.
    """
    for i in range(0, len(text), line_length):
        print(text[i:i + line_length])

# Example usage:
#long_text = "This is a very long string that needs to be wrapped so that it does not exceed a certain number of characters per line. We will demonstrate how to achieve this using Python."


In [7]:
print_wrapped_text(str(total_poly))


e**12*q**5 + 12*e**11*f*q**5 + 12*e**11*g*q**4 + 6*e**11*h*q**6 + 3*e**10*f**2*q**6 + 48*e**10*f**2*
q**5 + 6*e**10*f*g*q**5 + 108*e**10*f*g*q**4 + 60*e**10*f*h*q**6 + 6*e**10*g**2*q**4 + 48*e**10*g**2
*q**3 + 66*e**10*g*h*q**5 + 15*e**10*h**2*q**7 + 12*e**9*f**3*q**6 + 76*e**9*f**3*q**5 + 42*e**9*f**
2*g*q**5 + 336*e**9*f**2*g*q**4 + 6*e**9*f**2*h*q**7 + 198*e**9*f**2*h*q**6 + 84*e**9*f*g**2*q**4 + 
312*e**9*f*g**2*q**3 + 12*e**9*f*g*h*q**6 + 516*e**9*f*g*h*q**5 + 120*e**9*f*h**2*q**7 + 48*e**9*g**
3*q**3 + 64*e**9*g**3*q**2 + 24*e**9*g**2*h*q**5 + 252*e**9*g**2*h*q**4 + 150*e**9*g*h**2*q**6 + 20*
e**9*h**3*q**8 + 17*e**8*f**4*q**6 - 19*e**8*f**4*q**5 + 60*e**8*f**3*g*q**5 + 360*e**8*f**3*g*q**4 
+ 16*e**8*f**3*h*q**7 + 208*e**8*f**3*h*q**6 + 9*e**8*f**2*g**2*q**5 + 224*e**8*f**2*g**2*q**4 + 716
*e**8*f**2*g**2*q**3 + 44*e**8*f**2*g*h*q**6 + 1322*e**8*f**2*g*h*q**5 + 4*e**8*f**2*h**2*q**8 + 292
*e**8*f**2*h**2*q**7 + 18*e**8*f*g**3*q**4 + 300*e**8*f*g**3*q**3 + 288*e**8*f*g**3*q**2 + 

In [17]:
total_poly.subs({f:0, e:0})

g**6*h**6*q**8 - 5*g**6*h**6*q**7 + 10*g**6*h**6*q**6 - 7*g**6*h**6*q**5

In [16]:
import numpy as np 
from sympy import Matrix, nroots, symbols
from sympy import S

x, y, Q= symbols('x y Q')
q, e, f, g, h = symbols('q e f g h')
for n in [3, 4, 5, 6, 11, 21, 31, 41, 51]:
    x = S(1)/S(99)
    y = S(99)
    start = 0
    end = 2
    for i in range(50, 150):

        Q = S(i)/S(100)
    
        polyq = total_poly.subs({q:Q})
        M = Matrix([[x, 0, 0, 0], [y,x + y + x*y, 0, y + (Q**(-1))*x*y],  [x*y*(Q**(-1)), y+x*y*Q**(-1), x+y+x*y, 0], [Q, Q, 0, Q+x]])

        N = Matrix([[Q**2*x, Q**2*y,Q*(2*x*y + y**2 + x*y**2), Q**3]]).transpose()
        M_n = M**n*N
        a, b, c, d = M_n[0]/Q**2, M_n[1]/Q**2, M_n[2]/Q, M_n[3]/Q**3

    

        X = polyq.subs({e: a, f: b, g: c, h: d}).expand()
        if ((X < 0) and (start == 0)):
            start = float(i/100)
        if (X < 0):
            end = float(i/100)
    print("For the graph gadget G_",n, "the interval where the difference is negative is [",start,",", end, "]")
    


For the graph gadget G_ 3 the interval where the difference is negative is [ 0.7 , 1.08 ]
For the graph gadget G_ 4 the interval where the difference is negative is [ 0.62 , 1.25 ]
For the graph gadget G_ 5 the interval where the difference is negative is [ 0.59 , 1.32 ]
For the graph gadget G_ 6 the interval where the difference is negative is [ 0.58 , 1.36 ]
For the graph gadget G_ 11 the interval where the difference is negative is [ 0.56 , 1.42 ]
For the graph gadget G_ 21 the interval where the difference is negative is [ 0.56 , 1.42 ]
For the graph gadget G_ 31 the interval where the difference is negative is [ 0.56 , 1.43 ]
For the graph gadget G_ 41 the interval where the difference is negative is [ 0.56 , 1.43 ]
For the graph gadget G_ 51 the interval where the difference is negative is [ 0.56 , 1.43 ]
